# model_2: actually playing the game

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import sys

In [5]:
from google.colab import drive

drive.mount('/content/drive')
RunningInCOLAB = 'google.colab' in str(get_ipython())

path = "./data_collection/snake2_output.csv"
if RunningInCOLAB:
  path = "/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/data_collection/training_data/snake2_game_output_1.csv"
  sys.path.append('/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_import = pd.read_csv(path)
df_import.head()

# place holder columns for food locations
df_import['foodX'] = 0
df_import['foodY'] = 0

df = df_import
df.head()

,obstacleL,obstableF,obstacleR,direction_choice,game_over,foodX,foodY
0,0,0,0,2,0,0,0
1,0,0,0,2,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0


In [6]:
# selecting better training set:
df_game_over = df[df['game_over'] == 1]
df_game_not_over = df[df['game_over'] == 0]
df_game_not_over_sample = df_game_not_over.sample(n=500, random_state=42)
df_game_not_over_sample

# this df will have more equal amounts of dead/alive examples
df_sample = pd.concat([df_game_over, df_game_not_over_sample], ignore_index=True)


In [7]:
# downsizing data so ratio of death/non-death is more euquak
df = df_sample

df['direction_choice'] = df['direction_choice'] - 1

In [8]:
X = df.drop('game_over', axis=1).to_numpy()
y = df['game_over'].to_numpy()
print(X.shape, "\n", y.shape)

X_feature_shape = X.shape[1]
X_feature_shape

(760, 6) 
 (760,)


6

In [17]:
# create testing and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
# init model
model = keras.Sequential()

# input layer
model.add(Input(shape=(X_feature_shape,)))
model.add(Dense(units=(5), activation='sigmoid'))
# model.add(Dense(units=(5)))
model.add(Dense(units=(1), activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=1e-1)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 5)                 35        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 41 (164.00 Byte)
Trainable params: 41 (164.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
obs = [1,0,0]
dir = 1
food = [0,0]

input = np.concatenate([obs, [dir], food]).reshape((1,6))

model.predict(input)

1/1 [==============================] - 0s 159ms/step


array([[0.26850748]], dtype=float32)

In [26]:
history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/40
17/17 [==============================] - 1s 16ms/step - loss: 0.1758 - accuracy: 0.8064 - val_loss: 0.1153 - val_accuracy: 0.8553
Epoch 2/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0801 - accuracy: 0.9154 - val_loss: 0.0489 - val_accuracy: 0.9474
Epoch 3/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0582 - accuracy: 0.9286 - val_loss: 0.0398 - val_accuracy: 0.9561
Epoch 4/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0472 - accuracy: 0.9474 - val_loss: 0.0358 - val_accuracy: 0.9781
Epoch 5/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0409 - accuracy: 0.9662 - val_loss: 0.0285 - val_accuracy: 0.9781
Epoch 6/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0369 - accuracy: 0.9662 - val_loss: 0.0259 - val_accuracy: 0.9781
Epoch 7/40
17/17 [==============================] - 0s 5ms/step - loss: 0.0341 - accuracy: 0.9680 - val_loss: 0.0244 - val_accuracy: 0.9781
Epoch 8/40
17/17 [=

In [27]:
history.history['accuracy']

[0.8063910007476807,
 0.9154135584831238,
 0.9285714030265808,
 0.9473684430122375,
 0.9661654233932495,
 0.9661654233932495,
 0.9680451154708862,
 0.969924807548523,
 0.969924807548523,
 0.969924807548523,
 0.969924807548523,
 0.969924807548523,
 0.969924807548523,
 0.9887217879295349,
 0.9981203079223633,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [38]:
# import game logic
from game import snake_game_2

def angle_between_two_points(p1: np.array, p2: np.array):
  # Calculate the dot product of the two vectors
  dot_product = np.dot(p1, p2)

  # Calculate the magnitudes of the vectors
  magnitude1 = np.linalg.norm(p1)
  magnitude2 = np.linalg.norm(p2)

  # Calculate the cosine of the angle between the vectors
  cosine_angle = dot_product / (magnitude1 * magnitude2)

  # Calculate the angle in radians
  angle_radians = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

  # Convert radians to degrees
  angle_degrees = np.degrees(angle_radians)

  return angle_degrees

def score_move(obs, direction, food_location, head_location):
  """
  Custom loss function for RL space
  Params:
    - move: [obstacleL, obstacleF, obstalceR, direction, foodX, foodY]
  """
  # higher numbers are WORSE

  death = 0

  if move_will_kill_snake(obs, direction):
    death = 1

  # calulate angle between snake and food
  food_angle = angle_between_two_points(np.array(food_location), np.array(head_location))

  WEIGHT_food = 2
  WEIGHT_death = 30

  # print(food_angle)

  return abs(food_angle)*WEIGHT_food + death*WEIGHT_death

def move_will_kill_snake(state: np.array, direction: int):
  """Check whether or not move will kill snake"""
  return state[direction + 1] == 1

EPOCHS: int = 1

for _ in range(EPOCHS):
  game = snake_game_2.Game(gui_enabled=False)

  snake_dead = False

  while not snake_dead:
    obs = game.generate_observation()
    food_location = game.get_food_coordinates()

    head_location = game.get_snake_head_coordinates()

    # print(food_location)
    score_predictions = []
    score_actual = []
    states = []

    for direction in [-1, 0, 1]:
        state = np.concatenate([obs, [direction], food_location]).reshape((1, 6))

        score_prediction = model.predict(state, verbose=0)

        actual_score = score_move(obs, direction, food_location, head_location)

        score_predictions.append(score_prediction)
        score_actual.append(actual_score)
        states.append(state)

    direction_choice = np.random.choice(np.where(np.array(score_predictions) < 1)[0]) - 1

    game.change_direction_choose(direction_choice)

    X_train = np.array(states).reshape((3,6))
    y_train = np.array(score_actual)

    model.fit(X_train, y_train, batch_size=3)

    game.move()

    print(actual_score)





1/1 [==============================] - 0s 14ms/step - loss: 1.0000 - accuracy: 0.0000e+00
1.7075472925031877e-06
1/1 [==============================] - 0s 35ms/step - loss: 88.1509 - accuracy: 0.0000e+00
10.3888578154696
1/1 [==============================] - 0s 23ms/step - loss: 321.2934 - accuracy: 0.0000e+00
18.924644416051226
1/1 [==============================] - 0s 7ms/step - loss: 870.8548 - accuracy: 0.0000e+00
30.51023740611553
1/1 [==============================] - 0s 7ms/step - loss: 1815.0013 - accuracy: 0.0000e+00
43.602818972703666
1/1 [==============================] - 0s 8ms/step - loss: 2293.2532 - accuracy: 0.0000e+00


Exception ignored in: <function _xla_gc_callback at 0x7be99a30dc60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


48.88790956083306
1/1 [==============================] - 0s 8ms/step - loss: 2717.5493 - accuracy: 0.0000e+00
53.130102354156016
1/1 [==============================] - 0s 7ms/step - loss: 3091.5300 - accuracy: 0.0000e+00
56.60151153201277
1/1 [==============================] - 0s 9ms/step - loss: 2060.9055 - accuracy: 0.0000e+00
46.397181027296405
1/1 [==============================] - 0s 8ms/step - loss: 1286.6505 - accuracy: 0.0000e+00
36.869897645844034
1/1 [==============================] - 0s 8ms/step - loss: 1608.9811 - accuracy: 0.0000e+00
41.112090439166955
1/1 [==============================] - 0s 8ms/step - loss: 1914.9625 - accuracy: 0.0000e+00
44.76027010391917
1/1 [==============================] - 0s 8ms/step - loss: 2717.5493 - accuracy: 0.0000e+00
53.13010235415599
1/1 [==============================] - 0s 7ms/step - loss: 3712.1638 - accuracy: 0.0000e+00
61.92751306414702
1/1 [==============================] - 0s 7ms/step - loss: 4910.5576 - accuracy: 0.0000e+00


KeyboardInterrupt: 

In [ ]:
# Create a figure with two subplots arranged in a single row
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))  # Adjust figsize as needed

# Plot training & validation loss values
ax1.plot(history.history['loss'])
ax1.plot(history.history['val_loss'])
ax1.set_title('Model loss')
ax1.set_ylabel('Loss')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation accuracy values
ax2.plot(history.history['accuracy'])
ax2.plot(history.history['val_accuracy'])
ax2.set_title('Model accuracy')
ax2.set_ylabel('Accuracy')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Validation'], loc='upper left')

# Display the plots
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()